In [1]:
!wget -c "https://onedrive.live.com/download?cid=A5E3C415BF70F5D8&resid=A5E3C415BF70F5D8%217785&authkey=AP5Xz9M015K8zbc" -O "onnx_partial.tar.xz"

--2022-04-10 14:23:45--  https://onedrive.live.com/download?cid=A5E3C415BF70F5D8&resid=A5E3C415BF70F5D8%217785&authkey=AP5Xz9M015K8zbc
Resolving onedrive.live.com (onedrive.live.com)... 52.179.121.14
Connecting to onedrive.live.com (onedrive.live.com)|52.179.121.14|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://piqdcg.db.files.1drv.com/y4mf7FuJkaCViQjgvqZSHZ7GB7GnlHq5djTloAceDG1CQA72AbI-gc_C1nUj417i4wem1WiUHTTbEMgqUoVLuhrWhOGP8RNSeZ3UVdaXgEB_rgW_ZIplwX4b6ms2Eb3-Vx0oUtuiermyk_4RO_O6Dm-Cx9yhQBF0MDgP5SQX8CHbpeeJ0VC_X8D3DFPETx0EFmKhPQJDbVj4C_zamTDuXJr0A/onnx_partial.tar.xz?download&psid=1 [following]
--2022-04-10 14:23:46--  https://piqdcg.db.files.1drv.com/y4mf7FuJkaCViQjgvqZSHZ7GB7GnlHq5djTloAceDG1CQA72AbI-gc_C1nUj417i4wem1WiUHTTbEMgqUoVLuhrWhOGP8RNSeZ3UVdaXgEB_rgW_ZIplwX4b6ms2Eb3-Vx0oUtuiermyk_4RO_O6Dm-Cx9yhQBF0MDgP5SQX8CHbpeeJ0VC_X8D3DFPETx0EFmKhPQJDbVj4C_zamTDuXJr0A/onnx_partial.tar.xz?download&psid=1
Resolving piqdcg.db.files.1drv.com (piqdcg.d

In [2]:
!mkdir /content/models/
!tar -xf onnx_partial.tar.xz -C /content/models/

In [3]:
!pip install onnxruntime

     |████████████████████████████████| 5.2 MB 9.5 MB/s 


In [4]:
!pip install torch

In [5]:
import onnxruntime
import numpy as np
import os
import glob
import pandas as pd
from tqdm import tqdm
import PIL.Image as Image

In [6]:
onnx_path="/content/models/onnx_zoo/alexnet_places365_11.onnx"

In [7]:
!wget -c "https://db3pap006files.storage.live.com/y4mrPKskxLov4fA-5HlyV5_0B0bM1p9ZBjvbZpmZOG6slVD0SHK4h-TarKzo_RDnltFHjfZBQrlTzDY6Y8QT5-vKgERJfxWa9K3rRXPhKI5BKwQ9RHCvMf5dCvZximq8XGA0RTkFtpKGBMPkcQ7BON1PeYzjgGtVazIbMD9hTFhUolYgFajfFQp2nflggb-dqE-?width=1024&height=768&cropmode=none" -O "testimage.jpg"

--2022-04-10 17:19:29--  https://db3pap006files.storage.live.com/y4mrPKskxLov4fA-5HlyV5_0B0bM1p9ZBjvbZpmZOG6slVD0SHK4h-TarKzo_RDnltFHjfZBQrlTzDY6Y8QT5-vKgERJfxWa9K3rRXPhKI5BKwQ9RHCvMf5dCvZximq8XGA0RTkFtpKGBMPkcQ7BON1PeYzjgGtVazIbMD9hTFhUolYgFajfFQp2nflggb-dqE-?width=1024&height=768&cropmode=none
Resolving db3pap006files.storage.live.com (db3pap006files.storage.live.com)... 13.104.208.165
Connecting to db3pap006files.storage.live.com (db3pap006files.storage.live.com)|13.104.208.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184444 (180K) [image/jpeg]
Saving to: ‘testimage.jpg’

testimage.jpg       100%[===================>] 180.12K   336KB/s    in 0.5s    

2022-04-10 17:19:31 (336 KB/s) - ‘testimage.jpg’ saved [184444/184444]



In [8]:
img_path = "/content/testimage.jpg"

In [9]:
def process_image(img_path,input_shape):
    img = Image.open(img_path).convert("RGB")
    img = img.resize(input_shape)
    image = np.array(img, dtype=np.float32)
    image = image.transpose((2,0,1))[np.newaxis, ...]
    return image

In [10]:
def testOnnxRuntime(path):
  result=dict()

  result['name']=os.path.basename(path)[:-5]

  session = onnxruntime.InferenceSession(onnx_path)
  inname = [input.name for input in session.get_inputs()]
  outname = [output.name for output in session.get_outputs()]

  data_input = np.random.randn(*[input.shape for input in session.get_inputs()][0]).astype(np.float32)
  try:
    data_output = session.run(outname, {inname[0]: data_input})
    result['random_successful']=True
    result['error_random']="NaN"
  except Exception as e:
    result['random_successful']=False
    result['error_random']=str(e)

  data_input = process_image(img_path,tuple([input.shape for input in session.get_inputs()][0][2:4]))
  try:
    data_output = session.run(outname, {inname[0]: data_input})
    result['image_successful']=True
    result['error_image']="NaN"
  except Exception as e:
    result['image_successful']=False
    result['error_image']=str(e)

  return result

In [11]:
testOnnxRuntime(onnx_path)

{'error_image': 'NaN',
 'error_random': 'NaN',
 'image_successful': True,
 'name': 'alexnet_places365_11',
 'random_successful': True}

In [12]:
paths = glob.glob('/content/models/onnx_zoo/**/*.onnx', recursive=True)

In [13]:
len(paths)

487

In [14]:
df=pd.DataFrame([testOnnxRuntime(path) for path in tqdm(paths)])
df

100%|██████████| 487/487 [04:10<00:00,  1.95it/s]


,name,random_successful,error_random,image_successful,error_image
0,unet_carvana_carvana_11,True,NaN,True,NaN
1,agripredict_disease_classification_prop_11,True,NaN,True,NaN
2,tinynet_c_imagenet_11,True,NaN,True,NaN
3,pix2pix_map2sat_g_11,True,NaN,True,NaN
4,cyclegan_apple2orange_g_11,True,NaN,True,NaN
...,...,...,...,...,...
482,squeezenet1_1_imagenet_11,True,NaN,True,NaN
483,resnet18_imagenet_11,True,NaN,True,NaN
484,squeezenet1_0_imagenet_11,True,NaN,True,NaN
485,densenet161_imagenet_11,True,NaN,True,NaN


In [15]:
df.to_csv("runtime_onnx_zoo.csv")

In [16]:
len(df[(df['random_successful']==True)])/len(df)*100

100.0

In [17]:
len(df[(df['image_successful']==True)])/len(df)*100

100.0